09_inference_pipeline.ipynb notebook

In [3]:
print("Test_df before FE:", test_df.shape)
print("Nulls in sentiment:", test_df['avg_sentiment_score'].isna().sum())
print(test_df.tail(20))


Test_df before FE: (502, 10)
Nulls in sentiment: 502
          Date        Open        High         Low       Close    Volume  \
482 2025-08-18  230.229996  231.910004  228.330002  231.490005  25248900   
483 2025-08-19  230.089996  230.529999  227.119995  228.009995  29891000   
484 2025-08-20  227.119995  227.270004  220.919998  223.809998  36604300   
485 2025-08-21  222.649994  222.779999  220.500000  221.949997  32140500   
486 2025-08-22  222.789993  229.139999  220.820007  228.839996  37315300   
487 2025-08-25  227.350006  229.600006  227.309998  227.940002  22633700   
488 2025-08-26  227.110001  229.000000  226.020004  228.710007  26105400   
489 2025-08-27  228.570007  229.869995  227.809998  229.119995  21254500   
490 2025-08-28  229.009995  232.710007  228.020004  231.600006  33679600   
491 2025-08-29  231.320007  231.809998  228.160004  229.000000  26199200   
492 2025-09-02  223.520004  226.169998  221.830002  225.339996  38843900   
493 2025-09-03  225.210007  227.169

In [4]:
# ============================================================
# 09_inference_pipeline (FIXED)
# Predict only on VALID sentiment windows
# Test window: 2023-09-01 → 2023-12-15
# ============================================================

import pandas as pd
import numpy as np
import joblib
import xgboost as xgb
import lightgbm as lgb

print("🔥 Loading trained models...")

# ------------------------------------------------------------
# Load artifacts
# ------------------------------------------------------------
scaler     = joblib.load("../models/tab_scaler.pkl")
xgb_model  = joblib.load("../models/xgb_final.json")
lgb_model  = joblib.load("../models/lgb_final.txt")
meta_model = joblib.load("../models/ensemble_final.pkl")

print("✅ Models loaded successfully!")


# ============================================================
# 1) Load merged dataset (same used in training)
# ============================================================

df = pd.read_csv("../data/processed/stocks_news_merged.csv")
df["Date"] = pd.to_datetime(df["Date"])

df = df.sort_values(["Ticker", "Date"]).reset_index(drop=True)

print("📊 Dataset shape:", df.shape)
print("📅 Date range:", df["Date"].min(), "→", df["Date"].max())


# ============================================================
# 2) FIXED TEST WINDOW — sentiment exists here
# ============================================================

TEST_START = "2023-09-01"
TEST_END   = "2023-12-15"

test_df = df[df["Date"].between(TEST_START, TEST_END)].copy()
test_df = test_df.sort_values(["Ticker", "Date"]).reset_index(drop=True)

print("\n🧪 TEST WINDOW:")
print("Start:", test_df["Date"].min())
print("End:  ", test_df["Date"].max())
print("Rows:", len(test_df))

# Check sentiment availability
print("\nMissing sentiment in test window:", test_df["avg_sentiment_score"].isna().sum())


# ============================================================
# 3) Feature Engineering (same as training)
# ============================================================

def engineer_features(df):
    df = df.copy()
    df = df.sort_values(["Ticker", "Date"]).reset_index(drop=True)

    # --- Base Returns ---
    df["Return"] = df.groupby("Ticker")["Close"].pct_change()

    df["Return_lag1"] = df.groupby("Ticker")["Return"].shift(1)
    df["Return_lag2"] = df.groupby("Ticker")["Return"].shift(2)
    df["Return_lag3"] = df.groupby("Ticker")["Return"].shift(3)

    # --- Rolling ---
    df["return_ma5"] = df.groupby("Ticker")["Return_lag1"].transform(lambda x: x.rolling(5).mean())
    df["Volatility"] = df.groupby("Ticker")["Return_lag1"].transform(lambda x: x.rolling(5).std())
    df["Volatility_10"] = df.groupby("Ticker")["Return_lag1"].transform(lambda x: x.rolling(10).std())

    df["price_mom5"] = df.groupby("Ticker")["Close"].pct_change(5)
    df["price_trend5"] = df.groupby("Ticker")["Close"].transform(lambda x: x.rolling(5).mean())

    # --- Sentiment ---
    df["sentiment_lag1"] = df.groupby("Ticker")["avg_sentiment_score"].shift(1)
    df["sentiment_lag2"] = df.groupby("Ticker")["avg_sentiment_score"].shift(2)
    df["sentiment_lag3"] = df.groupby("Ticker")["avg_sentiment_score"].shift(3)

    df["sentiment_ma3"] = df.groupby("Ticker")["avg_sentiment_score"].transform(lambda x: x.rolling(3).mean())
    df["sentiment_ma5"] = df.groupby("Ticker")["avg_sentiment_score"].transform(lambda x: x.rolling(5).mean())
    df["sentiment_std5"] = df.groupby("Ticker")["avg_sentiment_score"].transform(lambda x: x.rolling(5).std())

    df["sentiment_mom"]  = df.groupby("Ticker")["avg_sentiment_score"].diff(1)
    df["sentiment_mom2"] = df.groupby("Ticker")["avg_sentiment_score"].diff(2)

    df["sentiment_vol_interact"] = df["avg_sentiment_score"] * df["Volatility"]
    df["sentiment_return_interact"] = df["avg_sentiment_score"] * df["Return_lag1"]

    df["return_sent_corr"] = df.groupby("Ticker").apply(
        lambda g: g["Return_lag1"].rolling(5).corr(g["avg_sentiment_score"])
    ).reset_index(level=0, drop=True)

    # --- RSI ---
    def calc_rsi(series, window=10):
        delta = series.diff()
        gain = delta.clip(lower=0).rolling(window).mean()
        loss = -delta.clip(upper=0).rolling(window).mean()
        rs = gain / (loss + 1e-9)
        return 100 - (100 / (1 + rs))

    df["RSI_10"] = df.groupby("Ticker")["Close"].transform(calc_rsi)

    # Feature list (same as training)
    FEATURES = [
        "Return_lag1","Return_lag2","Return_lag3","return_ma5","Volatility","Volatility_10",
        "price_mom5","price_trend5",
        "sentiment_lag1","sentiment_lag2","sentiment_lag3",
        "sentiment_ma3","sentiment_ma5","sentiment_std5",
        "sentiment_mom","sentiment_mom2",
        "sentiment_return_interact","sentiment_vol_interact","return_sent_corr",
        "RSI_10"
    ]

    # Remove rows that lack required rolling-window history
    df = df.dropna(subset=FEATURES).reset_index(drop=True)

    return df, FEATURES


# ------------------------------------------------------------
# Apply FE to test window
# ------------------------------------------------------------
test_fe, FEATURES = engineer_features(test_df)

print("\n✨ After FE:")
print("Table shape:", test_fe.shape)
print("Feature count:", len(FEATURES))


# ============================================================
# 4) Make Predictions
# ============================================================

X = test_fe[FEATURES].values
X_scaled = scaler.transform(X)

xgb_prob = xgb_model.predict_proba(X_scaled)[:, 1]
lgb_prob = lgb_model.predict_proba(X_scaled)[:, 1]

meta_input = np.column_stack([xgb_prob, lgb_prob])
ensemble_prob = meta_model.predict_proba(meta_input)[:, 1]

test_fe["Ensemble_Prob"] = ensemble_prob
test_fe["Prediction"]    = (ensemble_prob > 0.5).astype(int)

print("🎯 Predictions complete!")
test_fe.head()


# ============================================================
# 5) Save Output for Streamlit Dashboard
# ============================================================

output_path = "../data/testing_predictions_clean.csv"
test_fe.to_csv(output_path, index=False)

print("\n💾 Saved predictions to:", output_path)
print("🎉 Streamlit dashboard can now use EXACT matching dates (2023 only!)")


🔥 Loading trained models...
✅ Models loaded successfully!
📊 Dataset shape: (2510, 10)
📅 Date range: 2020-09-16 00:00:00 → 2025-09-15 00:00:00

🧪 TEST WINDOW:
Start: 2023-09-01 00:00:00
End:   2023-12-15 00:00:00
Rows: 148

Missing sentiment in test window: 4

✨ After FE:
Table shape: (116, 31)
Feature count: 20
🎯 Predictions complete!

💾 Saved predictions to: ../data/testing_predictions_clean.csv
🎉 Streamlit dashboard can now use EXACT matching dates (2023 only!)


C:\Users\User\AppData\Local\Temp\ipykernel_23260\3608975919.py:96: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df["return_sent_corr"] = df.groupby("Ticker").apply(
